In [ ]:
import os
import pandas as pd
import openpyxl
from copy import copy

In [ ]:
# print('The openpyxl version is {}.'.format(openpyxl.__version__))
# print('The pandas version is {}.'.format(pd.__version__))

In [ ]:
os.chdir(r'C:\Users\edalata\OneDrive - Origin Energy\Peat\Reserves and Assurance\Reserve 2020\Scaling Calculation\Python')

In [ ]:
result_export_table = 'COMP_EXP_MONTHLY_HOR_WELLS_EDITED'
Excel_filename='TypeCurveLoader.xlsx'

### Initialising pandas DataFrame

In [ ]:
df = pd.read_table(result_export_table, header = 0)

In [ ]:
df.columns = [x.strip() for x in df.columns]
df.rename(columns={'Unnamed: 0':'Dates'}, inplace=True)
df.index = df.loc[:]['Dates']
df.drop(['Dates'], axis = 1, inplace=True)

In [ ]:
properties = [x for x in df.iloc[0]]
current_clmns = [x for x in df.columns]
nw_clmns=[]
for i in range(len(properties)):
        if current_clmns[i][-2:] == ".1":
            current_clmns[i] = current_clmns[i][:-2]
        nw_clmns.append(current_clmns[i] + "_" + properties[i])
df.columns = nw_clmns

In [ ]:
df = df.iloc[1:]

In [ ]:
# created a list of wells by removing _GPR from the tale
wells = [x[:-4] for x in df.columns[:int(len(df.columns)/2)]]

### Initialising openPyXL

In [ ]:
wb = openpyxl.load_workbook(Excel_filename)
sheet = wb.active

### Creating a dictionray to covert well names from ENV to current GIS

In [ ]:
well_name_convertor = {}
with open('Lin2GISnameConverter.txt', 'r') as file:
    for line in file:
        line_data = line.split(',')
        well_name_convertor[line_data[0]] = line_data[1][:-1]

### Functions to iterate over, write and format data

In [ ]:
def writing_wells(well, well_index):
    strt_clmn = 2
    if well_index == 0:
        sheet.cell(row=1 , column=strt_clmn + well_index).value = well_name_convertor[well]
        sheet.cell(row=2 , column=strt_clmn + well_index).value = '1'
        sheet.cell(row=4 , column=strt_clmn + well_index).value = 'Initial drawdown 200 psia, then 0.75 psia per day.'
#         sheet.cell(row=5 , column=strt_clmn + well_index).value = 'Surface-In seam horizontal , 1500 m lateral length.'
#         sheet.cell(row=6 , column=strt_clmn + well_index).value = 'Fibreglass liner'
#         sheet.cell(row=7 , column=strt_clmn + well_index).value = '750 m .'
    else:
        sheet.cell(row=1 , column=strt_clmn + (well_index * 6)).value = well_name_convertor[well]
        sheet.cell(row=2 , column=strt_clmn + (well_index * 6)).value = '1'
        sheet.cell(row=4 , column=strt_clmn + (well_index * 6)).value = 'Initial drawdown 200 psia, then 0.75 psia per day.'
#         sheet.cell(row=5 , column=strt_clmn + (well_index * 6)).value = 'Surface-In seam horizontal , 1500 m lateral length.'
#         sheet.cell(row=6 , column=strt_clmn + (well_index * 6)).value = 'Fibreglass liner'
#         sheet.cell(row=7 , column=strt_clmn + (well_index * 6)).value = '750 m .'
    return None

In [ ]:
def writing_rates(well, well_index):
    strt_row = 13
    strt_clmn = 4
    rates = [well + '_GPR', well + '_WPR']
    
    for index,item in enumerate(rates):
        for i,v in enumerate([x for x in df.loc[:][item]]):
            sheet.cell(row=strt_row+i , column=strt_clmn + (well_index*6) + index).value = v
    return None

In [ ]:
def header_style(well_index):
    strt_clmn = 2
    
    for i in range(2):
        for j in range(6):
            if well_index > 0:
                master = sheet.cell(row=11 + i, column=strt_clmn + j)
                slave = sheet.cell(row=11 + i, column=strt_clmn + (well_index*6) + j)
            
                slave.value  = copy(master.value)
                slave.font   = copy(master.font)
                slave.border = copy(master.border)
                slave.fill   = copy(master.fill)
    return None

### Calling functions

In [ ]:
for well_index, well in enumerate(wells):
    writing_wells(well, well_index)
    writing_rates(well, well_index)
    header_style(well_index)

In [ ]:
wb.save(Excel_filename)
print('workbook successfully saved.')